# Microsoft Agent Framework Samples

Microsoft Agent Framework は、 .NET および Python 向けの AI エージェントおよびマルチエージェント ワークフロー を構築するためのオープンソース開発キットです。Semantic Kernel プロジェクト と AutoGen プロジェクト のアイデアを統合・拡張し、それぞれの長所を組み合わせながら新しい機能も追加しています。同じチームによって構築されたこのフレームワークは、今後の AI エージェント構築のための統一された基盤となります。



## Install

In [ ]:
!pip install agent-framework --pre

In [ ]:
from agent_framework.openai import OpenAIChatClient
from agent_framework.azure import AzureOpenAIChatClient
from dotenv import load_dotenv
import os

load_dotenv(override=True)

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
deployment_name = os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]

client = AzureOpenAIChatClient(
    deployment_name=deployment_name,
    endpoint=endpoint,
    api_key=api_key,
)

In [ ]:
client.to_json()

詳細な例については、以下を参照してください：

- [OpenAI Chat Client](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/agents/openai/openai_chat_client_basic.py) - Basic OpenAI client setup
- [Azure OpenAI Chat Client](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/agents/azure_openai/azure_chat_client_basic.py) - Azure OpenAI with authentication
- [Azure AI Client](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/agents/azure_ai/azure_ai_basic.py) - Azure AI agent integration

## Responses API Support
Agent Framework の `AzureOpenAIResponsesClient` および `OpenAIResponsesClient` は、AutoGen では利用できない推論モデルと構造化応答に対する専用サポートを提供します：

In [ ]:
from agent_framework.azure import AzureOpenAIResponsesClient
from agent_framework.openai import OpenAIResponsesClient

# Azure OpenAI with Responses API
azure_responses_client = AzureOpenAIResponsesClient(model_id="gpt-5")

# OpenAI with Responses API
# openai_responses_client = OpenAIResponsesClient(model_id="gpt-5")

Responses API の例については、以下を参照してください：

- [Azure Responses Client Basic](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/agents/azure_openai/azure_responses_client_basic.py) - Azure OpenAI with responses
- [OpenAI Responses Client Basic](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/agents/openai/openai_responses_client_basic.py) - OpenAI responses integration

## Observability
可観測性は、ワークフロー実行時の内部状態と動作に関する洞察を提供します。これには、ワークフローの監視とデバッグに役立つログ、メトリクス、トレース機能が含まれます。Agent Framework は OpenTelemetry のよりシンプルなゼロコードセットアップオプションを提供します。これによりすでに紹介した [Jaeger](https://qiita.com/nohanaga/items/6e0a42716e86eea58091) や [Azure AI Foundry](https://qiita.com/nohanaga/items/098d2bbdecac69bb3923) との連携による監視がより簡単にできるようになりました。さらにワークフローレベルの観測可能性を含むより広範なカバレッジを提供します。

エージェントのトレースはマルチエージェント開発には必須の機能と言っても過言ではないです。私のワークショップでも最初にトレースの設定を行います。


In [ ]:
from agent_framework.observability import setup_observability

setup_observability(
    enable_sensitive_data=True,
    otlp_endpoint="http://localhost:4317",
    # applicationinsights_connection_string="InstrumentationKey=your_key"
)

In [ ]:
from agent_framework.observability import get_tracer, get_meter

tracer = get_tracer()
meter = get_meter()

## Agent Hello World!

In [ ]:
from agent_framework import ChatAgent
# Agent Framework
with tracer.start_as_current_span("HelloWorld") as rollspan: # ルートスパンを作成

    agent = ChatAgent(name="assistant", chat_client=client)
    result = await agent.run("富士山の高さは?")

    print(result.text)

## ⭐️ AgentThread による会話状態の管理
単一エージェントの対話はステートレスです。`ChatAgent` との会話を続けるには、 `AgentThread` を使用して会話履歴を管理できるようになりました。これが最も基本的なコンテキスト共有の仕組みです。

In [ ]:
with tracer.start_as_current_span("HelloWorld") as rollspan: # ルートスパンを作成

    # スレッドなし（2つの独立した呼び出し）
    r1 = await agent.run("2+2は?")
    print(r1.text)  # e.g., "4"

    r2 = await agent.run("その数字に10を掛けるとどうなる？")
    print(r2.text)  # コンテキストがなければ曖昧な可能性あり；「40」とは限らない

In [ ]:
with tracer.start_as_current_span("HelloWorld") as rollspan: # ルートスパンを作成

    # スレッドを使用（呼び出し間で共有されるコンテキスト）
    thread = agent.get_new_thread()
    print((await agent.run("2+2は?", thread=thread)).text)  # "4"
    print((await agent.run("その数字に10を掛けるとどうなる？", thread=thread)).text)  # "40

In [ ]:
# メッセージの内容を詳しく見てみる
messages = await thread.message_store.list_messages()
print(f"メッセージ数: {len(messages)}")
print("=" * 50)

for i, msg in enumerate(messages):
    print(f"メッセージ {i + 1}:")
    print(f"  役割: {msg.role}")
    print(f"  テキスト: {msg.text}")
    print(f"  オブジェクト: {msg}")

In [ ]:
import json, pprint
# Serialize thread state
serialized_thread = await thread.serialize()

pprint.pprint(serialized_thread)
# Save to file/database
# with open("thread_state.json", "w") as f:
#     json.dump(serialized_thread, f) # 現状 ChatMessage オブジェクトのエラーが出てしまいます

## Tool Creation and Integration
ツールはテキスト生成を超えてエージェントの機能を拡張します。フレームワークはツール作成において異なるアプローチを取っており、Agent Framework はより自動化されたスキーマ生成を提供します。

Agent Framework は、エージェントレベルでツールの反復処理を自動で行います。AutoGen の`max_tool_iterations` パラメータとは異なり、Agent Framework のエージェントは、無限ループを防ぐための安全機構を組み込んでおり、デフォルトで完了までツールの実行を継続します。


In [ ]:
from agent_framework import ai_function
from typing import Annotated
from pydantic import Field

@ai_function
def get_weather(
    location: Annotated[str, Field(description="天気情報を入手するための場所を指定します")]
) -> str:
    """場所の天気情報を取得する。"""
    return f"Weather in {location}: rain"

# Direct use with agent (automatic conversion)
agent = ChatAgent(name="assistant", chat_client=client, tools=[get_weather])

with tracer.start_as_current_span("Tool1") as rollspan: # ルートスパンを作成
    # スレッドなし（2つの独立した呼び出し）
    response = await agent.run("東京の今日の天気は?")
    print(response)  # e.g., "4"

## Hosted Tools
Agent Framework は、AutoGen では利用できないホスト ツールを提供します。ホスト型ツールは、それらをサポートするモデル/アカウントでのみ利用できます。これらのツールを有効にする前に、プロバイダーの権限とモデルサポートを確認してください。以下は Azure AI Foundry Agent との連携。

In [ ]:
from agent_framework import ChatAgent, HostedCodeInterpreterTool, HostedWebSearchTool
from agent_framework.azure import AzureOpenAIChatClient

# Code execution tool
code_tool = HostedCodeInterpreterTool()

# Web search tool
search_tool = HostedWebSearchTool()

agent = ChatAgent(
    name="researcher",
    chat_client=client,
    tools=[code_tool, search_tool]
)

主な違い:
- デフォルト動作: ChatAgentはツール呼び出しを自動的に反復処理しますが、AssistantAgentは明示的なmax_tool_iterations設定が必要です
- 実行時設定: ChatAgent.run()は呼び出しごとのカスタマイズ用にtoolsおよびtool_choiceパラメータを受け付けます
- ファクトリメソッド: Agent Frameworkはチャットクライアントから直接便利なファクトリメソッドを提供します
- 状態管理: ChatAgentはステートレスであり、呼び出し間の会話履歴を維持しません。これに対しAssistantAgentは状態の一部として会話履歴を維持します

詳細な例については以下を参照してください:
- [Azure AI with Code Interpreter](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/agents/azure_ai/azure_ai_with_code_interpreter.py) - コード実行ツール
- [Azure AI with Multiple Tools](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/agents/azure_ai/azure_ai_with_multiple_tools.py) - 複数ホスト型ツール
- [OpenAI with Web Search](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/agents/openai/openai_chat_client_with_web_search.py) - Web検索統合

要件と注意点:
- ホスト型ツールは、それらをサポートするモデル/アカウントでのみ利用可能です。これらのツールを有効化する前に、プロバイダーの権限とモデルサポートを確認してください。
- プロバイダーごとに設定が異なります。セットアップと権限設定については、各サンプルの事前準備要件に従ってください。
- すべてのモデルがすべてのホスト型ツール（例: Web 検索とコード インタープリター）をサポートするわけではありません。環境で互換性のあるモデルを選択してください。

注記: AutoGen はローカルコード実行ツールをサポートしていますが、この機能は将来の Agent Framework バージョンで提供予定です。

主な違い: Agent Framework はエージェントレベルでツール反復処理を自動的に処理します。AutoGen の max_tool_iterations パラメータとは異なり、Agent Framework エージェントはデフォルトでツール実行を完了まで継続し、無限ループを防ぐ組み込みの安全メカニズムを備えています。

## MCP Server Support
高度なツール統合のため、両フレームワークはモデルコンテキストプロトコル（MCP）をサポートし、エージェントが外部サービスやデータソースと相互作用することを可能にします。Agent Framework はより包括的な組み込みサポートを提供します。


In [ ]:
from agent_framework import ChatAgent, MCPStdioTool, MCPStreamableHTTPTool, MCPWebsocketTool

# Stdio MCP server
mcp_tool = MCPStdioTool(
    name="filesystem",
    command="uvx mcp-server-filesystem",
    args=["/allowed/directory"]
)

# HTTP streaming MCP
http_mcp = MCPStreamableHTTPTool(
    name="http_mcp",
    url="https://learn.microsoft.com/api/mcp"
)

# WebSocket MCP
ws_mcp = MCPWebsocketTool(
    name="websocket_mcp",
    url="ws://localhost:8000/ws"
)

agent = ChatAgent(name="assistant", chat_client=client, tools=[http_mcp])

In [ ]:
with tracer.start_as_current_span("MCPChat") as rollspan: # ルートスパンを作成
    # Without a thread (two independent invocations)
    response = await agent.run("Azure AI Search の2025年の最新のニュースを教えてください")
    print(response.text)

詳細な例については、以下を参照してください：

- [ローカルMCPを使用したOpenAI](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/agents/openai/openai_chat_client_with_local_mcp.py) - OpenAIでMCPStreamableHTTPToolを使用する
- [ホスト型MCPを使用したOpenAI](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/agents/openai/openai_responses_client_with_hosted_mcp.py) - ホスト型MCPサービスを使用する
- [ローカルMCPを使用したAzure AI](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/agents/azure_ai/azure_ai_with_local_mcp.py) - Azure AIでMCPを使用する
- [ホスト型MCPを使用したAzure AI](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/agents/azure_ai/azure_ai_with_hosted_mcp.py) - Azure AIでホスト型MCPを使用する


## Agent-as-a-Tool Pattern
新たなマルチエージェントの強力なパターンの一つは、エージェント自体をツールとして使用し、階層的なエージェントアーキテクチャを実現することです。シンプルなマルチエージェントパターンを実装したい場合は、この機能で実現できます。

In [ ]:
from agent_framework import ChatAgent

# Create specialized agent
writer = ChatAgent(
    name="writer",
    chat_client=client,
    instructions="あなたは創造的で破天荒な作家です。"
)

# Convert to tool
writer_tool = writer.as_tool(
    name="creative_writer",
    description="創造的なコンテンツを生成する",
    arg_name="request",
    arg_description="何を書くか"
)

# Use in coordinator
coordinator = ChatAgent(
    name="coordinator",
    chat_client=client,
    tools=[writer_tool]
)

In [ ]:
with tracer.start_as_current_span("Agent-as-a-Tool") as rollspan: # ルートスパンを作成
    # Without a thread (two independent invocations)
    response = await coordinator.run("SEが異世界に転生して無双する話を書いてください")
    print(response.text) 

In [ ]:
result = response.to_dict()
result

## Middleware (Agent Framework Feature)
Agent FrameworkはAutoGenに欠けているミドルウェア機能を導入します。ミドルウェアにより、ロギング、セキュリティ、パフォーマンス監視といった強力な横断的関心事の処理が可能になります。

以下の `security_middleware` の例は関数呼び出しの引数に"password"が含まれている場合に実行をブロックします。以下のテストでこの動作を確認できます：


In [ ]:
from agent_framework import ChatAgent, AgentRunContext, FunctionInvocationContext
from typing import Callable, Awaitable
# パターン1: 直接的にパスワードを含む引数でツールを呼び出す

# 以前の例からクライアントがあると仮定する
async def logging_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]]
) -> None:
    print(f"Agent {context.agent.name} starting")
    await next(context)
    print(f"Agent {context.agent.name} completed")

async def security_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]]
) -> None:
    if "password" in str(context.arguments):
        print("機密データを含む関数呼び出しのブロック")
        return  # Don't call next()
    await next(context)

@ai_function
def test_security_function(input_text: str) -> str:
    """テスト用の関数"""
    return f"処理結果: {input_text}"

# セキュリティミドルウェア付きのエージェントを作成
secure_agent = ChatAgent(
    name="secure_test_agent",
    chat_client=client,
    middleware=[logging_middleware, security_middleware],
    tools=[test_security_function]
)

In [ ]:
print("=== テスト1: 直接的なパスワード文字列 ===")
try:
    result = await secure_agent.run("test_security_functionを使って 'my password is 123' を処理してください")
    print(f"結果: {result.text}")
except Exception as e:
    print(f"エラー: {e}")


利点:
- セキュリティ: 入力検証とコンテンツフィルタリング
- 可観測性: ログ記録、メトリクス、トレース
- パフォーマンス: キャッシュとレート制限
- エラー処理: 段階的劣化と再試行ロジック

詳細なミドルウェアの例については以下を参照:
- [関数ベースのミドルウェア](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/middleware/function_based_middleware.py) - シンプルな関数ミドルウェア
- [クラスベースのミドルウェア](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/middleware/class_based_middleware.py) - オブジェクト指向ミドルウェア
- [例外処理ミドルウェア](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/middleware/exception_handling_with_middleware.py) - エラー処理パターン
- [共有状態ミドルウェア](https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/middleware/shared_state_middleware.py) - エージェント間での状態管理


## Memory
エージェントメモリは、エージェントが会話間でコンテキストを維持し、ユーザーの好みを記憶し、パーソナライズされた体験を提供するための重要な機能です。エージェントフレームワークは、単純なインメモリストレージから永続データベース、専用メモリサービスに至るまで、様々なユースケースに対応する複数のメモリ機構を提供します。

### Memory の種類
エージェントフレームワークは、短期記憶の一部としてチャット履歴を管理したり、長期記憶を抽出して保存しエージェントに注入するための拡張ポイントを提供したりするなど、様々なユースケースに対応するため、複数のメモリタイプをサポートしています。

- In-Memory Storage (Default): チャットスレッドのこと
- Persistent Message Stores
    - Built-in ChatMessageStore
    - Redis Message Store
    - Custom Message Store
- Context Providers (Dynamic Memory)
    - External Memory Services(Mem0)



### Persistent Message Stores
セッション間で会話履歴を保持する必要のあるアプリケーションの場合、`ChatMessageStore` フレームワークは次の実装を提供します。


### 組み込みの ChatMessageStore
シリアライズ可能なデフォルトのインメモリ実装です。

In [ ]:
from agent_framework import ChatMessageStore, ChatAgent, AgentThread

# 方法1: スレッドから message_store を取得する
def create_message_store():
    return ChatMessageStore()

agent = ChatAgent(
    chat_client=client,
    instructions="You are a helpful travel assistant.",
    chat_message_store_factory=create_message_store
)

# スレッドを作成（この時点で新しいストアが作成される）
thread = agent.get_new_thread()

response = await agent.run("マレーシア行きの飛行機の予約をしてもらえますか？", thread=thread)
print(response.text)


In [ ]:
# スレッドから message_store を取得
if thread.message_store:
    messages = await thread.message_store.list_messages()
    print("\n=== 会話履歴 ===")
    for msg in messages:
        print(f"{msg.role}: {msg.text}")

In [ ]:
# スレッドの状態をシリアル化して、後で使用できるように保存できるようにする。
serialized_thread = await thread.serialize()

# スレッドはデータベース、ファイル、または他のストレージメカニズムに保存でき、後で再度読み込むことができる。
print(f"Serialized thread: {serialized_thread}\n")

# ストレージから読み込んだ後にスレッドの状態をデシリアライズする。
resumed_thread = await agent.deserialize_thread(serialized_thread)


In [ ]:
# メッセージの内容を詳しく見てみる
messages = await resumed_thread.message_store.list_messages()
print(f"メッセージ数: {len(messages)}")
print("=" * 50)

for i, msg in enumerate(messages):
    print(f"メッセージ {i + 1}:")
    print(f"  役割: {msg.role}")
    print(f"  テキスト: {msg.text}")
    print(f"  オブジェクト: {msg}")


### Redis Message Store
現在は永続ストレージとして Redis が指定できますが、今後は CosmosDB や Azure AI Search が追加されることでしょう。


In [ ]:
from agent_framework.redis import RedisChatMessageStore

def create_redis_store():
    return RedisChatMessageStore(
        redis_url="redis://localhost:6379",
        thread_id="user_session_123",
        max_messages=100  # Keep last 100 messages
    )

agent = ChatAgent(
    chat_client=OpenAIChatClient(),
    instructions="You are a helpful assistant.",
    chat_message_store_factory=create_redis_store
)

## Context Providers (Dynamic Memory)
コンテキスト プロバイダーは、各エージェントの呼び出しの前に関連するコンテキストを挿入することで、洗練されたメモリ パターンを実現します。メモリ設計については以下のスコープに分類できます。

| スコープ | 説明 | ユースケース |
|---------|------|------------|
| **グローバル** | 全スレッドで共有 | ユーザープロファイル、基本設定 |
| **スレッド単位** | スレッドごとに独立 | プロジェクト固有の情報 |
| **エージェント単位** | エージェントごとに独立 | パーソナル/ビジネス コンテキストの分離 |


### Mem0Provider

In [ ]:
from agent_framework.mem0 import Mem0Provider

# Using Mem0 for advanced memory capabilities
memory_provider = Mem0Provider(
    api_key="your-mem0-api-key",
    user_id="user_123",
    application_id="my_app"
)

agent = ChatAgent(
    chat_client=OpenAIChatClient(),
    instructions="You are a helpful assistant with memory.",
    context_providers=memory_provider
)

Mem0 についてはすでに Azure AI Search との連携を AutoGen で実現していましたね。メモリ機能についてはさらなる探究が必要です。

https://qiita.com/nohanaga/items/37fbe8dfb35a506b1d5a


# Multi-Agent Orchestration(WorkFlow)
## AutoGen のマルチエージェントパターンの移行状況

![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/674344/4d491a62-2ce2-43f2-a7b1-1427a4d49c64.png)


AutoGen の課題:
- 低レベルモデル（Core API）は多くのユーザーにとって複雑すぎる
- 高レベルモデル（AgentChat API）は複雑な動作に対して制約となり得る
- 両モデルの橋渡しは実装の複雑さを増す

Agent Framework は、両方のアプローチの長所を組み合わせた単一のワークフロー抽象化を提供します。

### GraphFlow vs WorkFlow
Agent Framework のワークフロー抽象化は AutoGen の実験的機能 GraphFlow に着想を得ていますが、設計思想において重要な進化を遂げています。

- **GraphFlow**：制御フローベース。エッジは遷移を表し、メッセージは全エージェントにブロードキャストされる。遷移はブロードキャストされたメッセージ内容によって条件付けられる
- **WorkFlow**：データフローベース。メッセージは特定のエッジを経由してルーティングされ、エッジによって Executor が起動される。並行実行をサポートする。

GraphFlow はエージェントを条件付き遷移とブロードキャストを持つノードとしてモデル化します。ワークフローは型付きエッジで接続された Executor（エージェント、関数、サブワークフロー）をモデル化し、リクエスト/レスポンスによる一時停止やチェックポイント機能もサポートします。

| 観点 | AutoGen | Agent Framework|
|------|----------------------|--------------------------------|
| **基本パラダイム** | 制御フロー（GraphFlow） | データフロー（WorkFlow） |
| **設計思想** | 「誰が次に話すか」を制御 | 「データがどこへ流れるか」を定義 |
| **グラフの意味** | 実行順序の可能性を表現 | データの流れを表現 |
| **エッジの役割** | 条件付き状態遷移 | 型付きデータパス |
| **ノード種別** | エージェントのみ | エージェント + 関数 + サブワークフロー |
| **メッセージ配信** | ブロードキャスト<br>(全ノードが全メッセージを受信) | ポイントツーポイント<br>(エッジで定義されたパスのみ) |

https://qiita.com/nohanaga/items/60c66935d75d3dbc48e2

AutoGen の制御フローをデータフローに変えてきたあたり、Microsoft らしいというか・・・Azure ML Pipeline や Prompt flow の系譜を感じざるを得ない。


# ワークフロー基礎

**WorkflowBuilder**
- 汎用的で柔軟なワークフロー構築
- ファンアウト/ファンイン、集約ロジックを明示的に実装
- より細かい制御が可能

**Concurrent/Sequential/Magentic Builder**
- エージェント専用の簡潔な API
- 内部的にディスパッチャーとアグリゲーターを自動生成
- ボイラープレートコードが不要

## 1. WorkflowBuilder
### 1.1. Sequential + Conditional
Agent Framework のワークフローのノード（Executor）はエージェント、純粋関数、またはサブワークフローとなり得ます。**注目は `add_edge` できるのがエージェントに限らない**という点ですね。Executor によって任意の関数を噛ませることができるというのと、メッセージの送信をコントロールできるという点が AutoGen と異なります。


#### ワークフローの便利な可視化関数を先に定義

In [ ]:
from agent_framework import WorkflowViz
from IPython.display import SVG, display
import os

def visualize_workflow(workflow, filename="workflow_diagram", show_preview=True):
    """
    ワークフローのグラフ情報を出力し、SVGで保存し、プレビューする関数
    
    Args:
        workflow: 可視化するワークフローオブジェクト
        filename: 保存するファイル名（拡張子なし）
        show_preview: プレビューを表示するかどうか
    
    Returns:
        保存されたSVGファイルのパス
    """
    # WorkflowVizオブジェクトを作成
    viz = WorkflowViz(workflow)
    
    # 3. SVGファイルとして保存
    try:
        svg_path = viz.export(format="svg", filename=filename)
        print("=" * 60)
        print(f"✅ SVGファイルを保存しました: {svg_path}")
        print("=" * 60)
        print()
        
        # 4. SVGをプレビュー表示
        if show_preview and os.path.exists(svg_path):
            display(SVG(filename=svg_path))
        
        return svg_path
        
    except ImportError as e:
        print("❌ エラー: 'graphviz'パッケージがインストールされていません")
        print("インストール方法: pip install agent-framework[viz] --pre")
        print(f"詳細: {e}")
        return None
    except Exception as e:
        print(f"❌ エラーが発生しました: {e}")
        return None

In [ ]:
# Agent Framework Workflow — 条件付きロジックを備えた順次実行
from agent_framework import WorkflowBuilder, executor, WorkflowContext
from typing_extensions import Never

@executor(id="writer")
async def writer_exec(task: str, ctx: WorkflowContext[str]) -> None:
    await ctx.send_message(f"Draft: {task}")

@executor(id="reviewer")
async def reviewer_exec(draft: str, ctx: WorkflowContext[str]) -> None:
    print(f"reviewer: message: {draft}")
    decision = "approve" if "solar" in draft.lower() else "revise"
    await ctx.send_message(f"{decision}:{draft}")

@executor(id="editor")
async def editor_exec(msg: str, ctx: WorkflowContext[Never, str]) -> None:
    print(f"editor: message: {msg}")
    if msg.startswith("approve:"):
        await ctx.yield_output(msg.split(":", 1)[1])
    else:
        await ctx.yield_output("Needs revision")

workflow_seq = (
    WorkflowBuilder()
    .add_edge(writer_exec, reviewer_exec)
    .add_edge(reviewer_exec, editor_exec)
    .set_start_executor(writer_exec)
    .build()
)

In [ ]:
# 関数を使ってワークフローを可視化
svg_path = visualize_workflow(
    workflow_seq, 
    filename="sequential_conditional_workflow",
    show_preview=True
)

In [ ]:
with tracer.start_as_current_span("Workflow") as rollspan: # ルートスパンを作成
# 使用例（非同期コンテキスト内での使用例）
    result = await workflow_seq.run("新しい発電方法としての solar energy の可能性についての記事を書いてください")
    result = await workflow_seq.run("新しい発電方法としての water energy の可能性についての記事を書いてください")

#### 注目👀：型安全なメッセージルーティング
- 入力型: ハンドラーの第一引数の型アノテーションから自動検出
- 出力型: `ctx.send_message()` で送信するメッセージの型
- ワークフロー出力型: `ctx.yield_output()` で返す最終結果の型

In [ ]:
result

### 1.2. Fan‑out + Fan-in
並行処理パターンにおける **Fan-out (ファンアウト)** と **Fan-in (ファンイン/合流)** は、複数のタスクを並列実行し、その結果を統合する際の重要なパターンです。Agent Framework では、`add_fan_out_edges` と `add_fan_in_edges` が利用可能です。

In [ ]:
import asyncio

@executor(id="dispatcher")
async def dispatcher(task: str, ctx: WorkflowContext[str]) -> None:
    await ctx.send_message(task)  # ← target_id 指定なし

@executor(id="worker1")
async def worker1(text: str, ctx: WorkflowContext[str]) -> None:
    await asyncio.sleep(2)
    await ctx.send_message(f"Worker1 result: {text}")

@executor(id="worker2")
async def worker2(text: str, ctx: WorkflowContext[str]) -> None:
    await asyncio.sleep(5)
    await ctx.send_message(f"Worker2 result: {text}")

@executor(id="aggregator")
async def aggregator(results: list[str], ctx: WorkflowContext[Never, str]) -> None:
    # FanInEdgeGroup が自動的にリストに集約
    await ctx.yield_output("Aggregated: " + " | ".join(results))

# Fan-out と Fan-in の組み合わせ
wf_combined = (
    WorkflowBuilder()
    .add_fan_out_edges(dispatcher, [worker1, worker2])  # dispatcher → [worker1, worker2]
    .add_fan_in_edges([worker1, worker2], aggregator)   # [worker1, worker2] → aggregator
    .set_start_executor(dispatcher)
    .build()
)

In [ ]:
# 関数を使ってワークフローを可視化
svg_path = visualize_workflow(
    wf_combined, 
    filename="fanout_fanin_workflow",
    show_preview=True
)

In [ ]:
with tracer.start_as_current_span("Workflow") as rollspan: # ルートスパンを作成
    result = await wf_combined.run("新しい発電方法としての solar energy の可能性についての記事を書いてください")


In [ ]:
result

**注目👀：実行フロー:**

- `dispatcher` が `task` を1回送信
- `FanOutEdgeGroup` が自動的に `worker1` と `worker2` に配信
- Sleep 後、両方が完了
- `FanInEdgeGroup` が結果を集約して `aggregator` を1回実行

### 1.3. Targeted Routing (no broadcast)
Targeted Routing は、Agent Framework のデータフロー型アーキテクチャにおける中核的な機能で、AutoGen のブロードキャスト方式とは根本的に異なるメッセージング戦略を採用しています。この機能により、メッセージを特定の宛先に直接送信することで、通信効率と予測可能性を大幅に向上させることができます。

In [ ]:
from agent_framework import WorkflowBuilder, executor, WorkflowContext
from typing_extensions import Never

@executor(id="ingest")
async def ingest(task: str, ctx: WorkflowContext[str]) -> None:
    # Route selectively using target_id
    if task.startswith("image:"):
        await ctx.send_message(task.removeprefix("image:"), target_id="vision")
    else:
        await ctx.send_message(task, target_id="writer")

@executor(id="writer")
async def write(text: str, ctx: WorkflowContext[Never, str]) -> None:
    await ctx.yield_output(f"Draft: {text}")

@executor(id="vision")
async def caption(image_ref: str, ctx: WorkflowContext[Never, str]) -> None:
    await ctx.yield_output(f"Caption: {image_ref}")

workflow = (
    WorkflowBuilder()
    .add_edge(ingest, write)
    .add_edge(ingest, caption)
    .set_start_executor(ingest)
    .build()
)


In [ ]:
# 関数を使ってワークフローを可視化
svg_path = visualize_workflow(
    workflow, 
    filename="conditional_routing_workflow",
    show_preview=True
)

In [ ]:
with tracer.start_as_current_span("Workflow") as rollspan: # ルートスパンを作成
    # Example usage (async):
    result = await workflow.run("Summarize the benefits of solar power")

In [ ]:
result

In [ ]:
with tracer.start_as_current_span("Workflow") as rollspan: # ルートスパンを作成
    # Example usage (async):
    result = await workflow.run("image:https://example.com/panel.jpg")

In [ ]:
result

`WorkflowBuilder` と Executor を組み合わせることで、これまでに無かった自由度の高さと決定論的データフローを実現しています。


## 2. ConcurrentBuilder
並列オーケストレーションにより、複数のエージェントが同一タスクを並行して処理できます。各エージェントは入力を独立して処理し、その結果が集約されます。この手法は、ブレインストーミング、アンサンブル推論、投票システムなど、多様な視点や解決策が価値を持つシナリオに最適です。

In [ ]:
from agent_framework import ConcurrentBuilder, WorkflowOutputEvent

agent1 = ChatAgent(
    name="writer",
    chat_client=client,
    instructions="あなたは創造的な小説家です。"
)

agent2 = ChatAgent(
    name="reviewer",
    chat_client=client,
    instructions="あなたは創造的なレビュアーです。インプットされた小説を批判的に評価します。"
)

agent3 = ChatAgent(
    name="editor",
    chat_client=client,
    instructions="あなたは細心の注意を払う編集者です。インプットされた小説を校正します。"
)

# 並列処理のための並行ワークフロー
workflow = (ConcurrentBuilder()
           .participants([agent1, agent2, agent3])
           .build())

# 使用例（非同期コンテキスト内での使用例）
async def concurrent_example():
    with tracer.start_as_current_span("ConcurrentBuilder") as rollspan: # ルートスパンを作成
        # すべてのエージェントが入力を同時に処理する
        async for event in workflow.run_stream("SEが異世界に転生して無双する話を書いてください"):
            if isinstance(event, WorkflowOutputEvent):
                results = event.data  # 全エージェントからの統合結果

                print("=== Concurrent Results ===")
                for i, res in enumerate(results):
                    print(f"Agent {i+1} result: {res.text}")
                    

In [ ]:
# 関数を使ってワークフローを可視化
svg_path = visualize_workflow(
    workflow, 
    filename="concurrent_workflow",
    show_preview=True
)

In [ ]:
# 使用例の実行
await concurrent_example()

トレースを見てすべてのエージェントが並列で処理されていることを確認してください。
このシナリオを並列で処理してしまうと問題があることが分かります。

**注目👀：内部で自動生成されるもの:**
- ディスパッチャー (入力を各エージェントに配信)
- 各エージェント用のラッパーExecutor
- アグリゲーター (全結果を集約)
- エッジ定義

複数のエージェントに同じ入力を並列処理させたい、全エージェントの結果を単純に集約したい、素早くプロトタイプを作りたい、エージェント専用の並列処理を作りたい場合 `ConcurrentBuilder` が適しています。



## 3. SequentialBuilder as Group Chat Patterns
順次オーケストレーションでは、エージェントはパイプラインに組織化されます。各エージェントは順番にタスクを処理し、その出力をシーケンス内の次のエージェントに渡します。これは、文書レビュー、データ処理パイプライン、多段階推論など、各ステップが前のステップを基盤とするワークフローに理想的です。


In [ ]:
from agent_framework import SequentialBuilder, WorkflowOutputEvent
from agent_framework import ChatAgent

# 以前の例から、エージェント1、エージェント2、エージェント3があると仮定
# Sequential workflow through participants
workflow = SequentialBuilder().participants([agent1, agent2, agent3]).build()

# 使用例（非同期コンテキスト内での使用例）
async def sequential_example():
    with tracer.start_as_current_span("SequentialBuilder") as rollspan: # ルートスパンを作成
        # 各エージェントは共有会話に追加する
        async for event in workflow.run_stream("SEが異世界に転生して無双する話を書いてください"):
            if isinstance(event, WorkflowOutputEvent):
                conversation_history = event.data  # list[ChatMessage]

In [ ]:
# 関数を使ってワークフローを可視化
svg_path = visualize_workflow(
    workflow, 
    filename="sequential_workflow",
    show_preview=True
)

In [ ]:
await sequential_example()

まぁ Group Chat パターンは Sequential フローですからね。。。この実装になると思います。


## 4. MagenticBuilder パターン
Magentic オーケストレーションは、AutoGen が開発した Magentic-One システムに基づいて設計されています。これは、動的なコラボレーションを必要とする複雑でオープンエンドなタスク向けに設計された、柔軟で汎用的なマルチエージェントパターンです。このパターンでは、専任の Magentic マネージャーが専門エージェントのチームを調整し、変化するコンテキスト、タスクの進捗状況、エージェントの能力に基づいて、次にどのエージェントが行動すべきかを選択します。


![image.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/674344/17e098bb-aca8-48b5-87bb-4dcef3bef87c.png)


Magentic マネージャーは、共有コンテキストを維持し、進捗状況を追跡し、ワークフローをリアルタイムに適応させます。これにより、システムは複雑な問題を細分化し、サブタスクを委任し、エージェント間の連携を通じてソリューションを反復的に改良することが可能になります。このオーケストレーションは、解決策のパスが事前に不明で、複数回の推論、調査、DeepResearch、計算が必要となるシナリオに特に適しています。

In [ ]:
from agent_framework import (
    MagenticBuilder, MagenticCallbackMode, WorkflowOutputEvent,
    MagenticCallbackEvent, MagenticOrchestratorMessageEvent, MagenticAgentDeltaEvent
)

researcher = ChatAgent(
    name="researcher",
    chat_client=client,
    instructions="You are a diligent researcher."
)

coder = ChatAgent(
    name="coder",
    chat_client=client,
    instructions="You are a skilled Python coder."
)

# 研究者、コーダー、およびクライアント調整役
async def on_event(event: MagenticCallbackEvent) -> None:
    if isinstance(event, MagenticOrchestratorMessageEvent):
        print(f"[ORCHESTRATOR]: {event.message.text}")

workflow = (MagenticBuilder()
           .participants(researcher=researcher, coder=coder)
           .on_event(on_event, mode=MagenticCallbackMode.STREAMING)
           .with_standard_manager(
               chat_client=client,
               max_round_count=20,
               max_stall_count=3,
               max_reset_count=2
           )
           .build())

# Example usage (would be in async context)
async def magentic_example():
    with tracer.start_as_current_span("MagenticBuilder") as rollspan: # ルートスパンを作成
        result = await workflow.run(
            "ブラックホールがどのように光を曲げるのかシミュレーションしたい。まず2Dで光線を描画するPythonコードを書いて。",
        )

        print("\n=== Magentic Final Result ===")
        print(result)


In [ ]:
# 関数を使ってワークフローを可視化
svg_path = visualize_workflow(
    workflow, 
    filename="magentic_workflow",
    show_preview=True
)

In [ ]:
await magentic_example()

In [ ]:
print(result.get_outputs()[0].text)

**注目👀：エージェントフレームワークのカスタマイズオプション:**
- **マネージャー設定**: カスタムオーケストレーターモデルとプロンプト
- **ラウンド制限**: `max_round_count`, `max_stall_count`, `max_reset_count`
- **イベントコールバック**：詳細なイベントフィルタリングによるリアルタイムストリーミング
- **エージェント特化**：エージェントごとのカスタム指示とツール
- **コールバックモード**：STREAMING（リアルタイム更新用）または BATCH（最終結果用）
- **Human-in-the-loop 計画**：対話型ワークフロー向けカスタムプランナー機能

https://qiita.com/nohanaga/items/28fcf00a23e990ac3551


### 🚧 将来のマルチエージェントパターン
現在以下のエージェントは開発中です。

- **Swarm**: ハンドオフベースのエージェントの調整
- **SelectorGroupChat**: LLM 駆動スピーカーの選択

## Workflow の可視化

Agent Framework のワークフローは、データフローグラフとして可視化できます。これにより、エージェント間の接続や処理の流れを視覚的に理解できます。

`WorkflowViz` ワークフローの可視化機能は、Workflow オブジェクトでインスタンス化できる `WorkflowViz` オブジェクトを通じて行われます。`WorkflowViz` オブジェクトは、Graphviz DOT 形式や Mermaid ダイアグラム形式など、様々な形式で可視化を生成できます。



In [ ]:
# ワークフローのグラフ構造を取得
graph_info = workflow.to_dict()
print("ワークフロー情報:")
import pprint
pprint.pprint(graph_info)

In [ ]:
from agent_framework import SequentialBuilder
from agent_framework import (
    WorkflowViz,
    handler,
)
# 2) Build sequential workflow: writer -> reviewer
workflow = SequentialBuilder().participants([agent1, agent2]).build()

viz = WorkflowViz(workflow)
# Print out the mermaid string.
print("Mermaid string: \n=======")
print(viz.to_mermaid())
print("=======")
# Print out the DiGraph string.
print("DiGraph string: \n=======")
print(viz.to_digraph())
print("=======")

In [ ]:
try:
    # Export the DiGraph visualization as SVG.
    svg_file = viz.export(format="svg", filename="sequential_workflow_diagram")
    print(f"SVG file saved to: {svg_file}")
except ImportError:
    print("Tip: Install 'viz' extra to export workflow visualization: pip install agent-framework[viz] --pre")


## 5. ⭐️ Shared States
Shared State により、ワークフロー内の複数の Executor が共通データにアクセスし、変更できるようになります。この機能は、ワークフローの複数の部分で情報を共有する必要があるものの、直接的なメッセージパッシングが実現不可能または効率的でないシナリオにおいて不可欠です。

In [ ]:
from agent_framework import WorkflowBuilder, executor, WorkflowContext
from typing_extensions import Never

# Executor 1: 共有状態にデータを保存
@executor(id="setter")
async def setter_exec(input_data: str, ctx: WorkflowContext[str]) -> None:
    # 共有状態にデータを保存
    await ctx.set_shared_state("user_name", "田中太郎")
    await ctx.set_shared_state("age", 30)
    await ctx.set_shared_state("city", "東京")
    
    print("✅ 共有状態に保存しました:")
    
    await ctx.send_message(input_data)

# Executor 2: 共有状態からデータを取得
@executor(id="getter")
async def getter_exec(data: str, ctx: WorkflowContext[Never, str]) -> None:
    # 共有状態からデータを取得
    user_name = await ctx.get_shared_state("user_name")
    age = await ctx.get_shared_state("age")
    city = await ctx.get_shared_state("city")
    
    print("\n📥 共有状態から取得しました:")
    print(f"  user_name: {user_name}")
    print(f"  age: {age}")
    print(f"  city: {city}")
    
    result = f"ユーザー情報: {user_name}さん、{age}歳、{city}在住"
    await ctx.yield_output(result)

# ワークフロー構築
workflow_shared_state = (
    WorkflowBuilder()
    .add_edge(setter_exec, getter_exec)
    .set_start_executor(setter_exec)
    .build()
)

# 実行
result = await workflow_shared_state.run("テストデータ")

print("\n" + "="*50)
print("🎯 最終結果:")
print(result)

## 6. ⭐️ Human-in-the-Loop with Request Response

Agent Framework のワークフローの主な新機能は 、要求と応答の概念です。これにより、ワークフローは実行を一時停止し、外部入力を待ってから続行できます。AutoGen の Team 抽象化は開始されると継続的に実行され、人間の入力の実行を一時停止するための組み込みのメカニズムは提供されませんでした。


In [ ]:
from agent_framework import (
    RequestInfoExecutor, RequestInfoEvent, RequestInfoMessage,
    RequestResponse, WorkflowBuilder, WorkflowContext, executor
)
from dataclasses import dataclass
from typing_extensions import Never

agent1 = ChatAgent(
    name="idea_agent",
    chat_client=client,
    instructions="あなたは創造的なアイデアライターです。"
)

# Workflow executor that generates content
@executor(id="writer")
async def agent_executor(
    task: str,
    ctx: WorkflowContext[str]
) -> None:
    response = await agent1.run(task)
    await ctx.send_message(response.text)


# Define typed request payload
@dataclass
class ApprovalRequest(RequestInfoMessage):
    """Request human approval for agent output."""
    content: str = ""
    agent_name: str = ""

# Workflow executor that requests human approval
@executor(id="reviewer")
async def approval_executor(
    agent_response: str,
    ctx: WorkflowContext[ApprovalRequest]
) -> None:
    # Request human input with structured data
    approval_request = ApprovalRequest(
        content=agent_response,
        agent_name="writer_agent"
    )
    await ctx.send_message(approval_request)

# Human feedback handler
@executor(id="processor")
async def process_approval(
    feedback: RequestResponse[ApprovalRequest, str],
    ctx: WorkflowContext[Never, str]
) -> None:
    decision = feedback.data.strip().lower()
    original_content = feedback.original_request.content

    if decision == "approve":
        await ctx.yield_output(f"APPROVED: {original_content}")
    else:
        await ctx.yield_output(f"REVISION NEEDED: {decision}")

# Build workflow with human-in-the-loop
hitl_executor = RequestInfoExecutor(id="request_approval")

workflow = (WorkflowBuilder()
           .add_edge(agent_executor, approval_executor)
           .add_edge(approval_executor, hitl_executor)
           .add_edge(hitl_executor, process_approval)
           .set_start_executor(agent_executor)
           .build())

In [ ]:
# 関数を使ってワークフローを可視化
svg_path = visualize_workflow(
    workflow, 
    filename="human_in_the_loop_workflow",
    show_preview=True
)

In [ ]:
# Example usage (would be in async context)
with tracer.start_as_current_span("HumanInTheLoop_Workflow") as rollspan: # ルートスパンを作成
    result = await workflow.run(
        "SEが異世界に転生して無双する話を書いてください",
    )

    print("\n=== HumanInTheLoop_Workflow Final Result ===")
    print(result)


## Human-in-the-Loop ワークフローの実行
Agent Framework には、一時停止再開サイクルを処理するためのストリーミング API が用意されています。


In [ ]:
from agent_framework import RequestInfoEvent, WorkflowOutputEvent

# 以前の例から定義されたワークフローがあると仮定する
async def run_with_human_input():
    pending_responses = None
    completed = False

    while not completed:
        # 最初の反復ではrun_streamを使用し、その後はsend_responses_streamingを使用する
        stream = (
            workflow.send_responses_streaming(pending_responses)
            if pending_responses
            else workflow.run_stream("initial input")
        )

        events = [event async for event in stream]
        pending_responses = None

        # 人間の要求と出力を収集する
        for event in events:
            print(f"イベント検知: {event}")
            if isinstance(event, RequestInfoEvent):
                # 人間に要求を表示し、応答を収集する
                request_data = event.data  # ApprovalRequest instance
                print(f"Review needed: {request_data.content}")

                human_response = input("Enter 'approve' or revision notes (or 'abort' to stop): ")
                # 中断処理
                if human_response.lower() == "abort":
                    print("\n🛑 ワークフローを中断しました")
                    completed = True
                    break
                
                pending_responses = {event.request_id: human_response}

            elif isinstance(event, WorkflowOutputEvent):
                print(f"Final result: {event.data}")
                completed = True

In [ ]:
await run_with_human_input()

### 全体的な Human-in-the-Loop ワークフローの実行例

[human-in-the-loop.py](./human-in-the-loop.py) をコンソールから実行してください。


## 7. ⭐️ Checkpoint
AutoGen の Team 抽象化に対する Agent Framework のワークフローのもう 1 つの主な利点は、**チェックポイント処理と実行の再開の組み込みサポート**です。 これにより、**ワークフローをチェックポイントから後で一時停止、永続化、再開できる**ため、フォールトトレランスが提供され、実行時間の長いワークフローまたは非同期ワークフローが有効になります。AutoGen の Team 抽象化では、組み込みのチェックポイント機能は提供されません。 永続化または回復メカニズムは外部で実装する必要があり、多くの場合、複雑な状態管理とシリアル化ロジックが必要です。


### 7.1. エージェント フレームワークのチェックポイント処理
Agent Framework では、`FileCheckpointStorage` と `WorkflowBuilder` の`with_checkpointing()` メソッドを使用した包括的なチェックポイント処理が提供されます。


In [ ]:
from agent_framework import (
    RequestInfoExecutor, FileCheckpointStorage, WorkflowBuilder, WorkflowOutputEvent,
    Executor, WorkflowContext, handler
)
from typing_extensions import Never

class UpperCaseExecutor(Executor):
    @handler
    async def process(self, text: str, ctx: WorkflowContext[str]) -> None:
        result = text.upper()

        # Executor固有の状態を永続化する
        prev_state = await ctx.get_state() or {}
        count = prev_state.get("count", 0) + 1
        await ctx.set_state({
            "count": count,
            "last_input": text,
            "last_output": result
        })

        # 他のExecutor向けに共有状態を永続化する
        await ctx.set_shared_state("original_input", text)
        await ctx.set_shared_state("processed_output", result)

        await ctx.send_message(result)

class ReverseExecutor(Executor):
    @handler
    async def process(self, text: str, ctx: WorkflowContext[Never, str]) -> None:
        result = text[::-1]
        await ctx.yield_output(result)

def create_workflow(checkpoint_storage: FileCheckpointStorage):
    """2つのExecutorとチェックポイント機能を備えたワークフローを作成する。"""
    upper_executor = UpperCaseExecutor(id="upper")
    reverse_executor = ReverseExecutor(id="reverse")

    return (WorkflowBuilder()
           .add_edge(upper_executor, reverse_executor)
           .set_start_executor(upper_executor)
           .with_checkpointing(checkpoint_storage=checkpoint_storage)
           .build())

checkpoint_storage = FileCheckpointStorage(storage_path="./checkpoints")

# 使用例（非同期コンテキスト内での使用例）
async def checkpoint_example():
    workflow = create_workflow(checkpoint_storage)
    async for event in workflow.run_stream("Hello, Agent Framework!"):
        if isinstance(event, WorkflowOutputEvent):
            print(f"Output: {event.data}")
    return workflow


In [ ]:
workflow = await checkpoint_example()
visualize_workflow(workflow, "Checkpointing_Workflow")

### 7.2. チェックポイントからの再開
Agent Framework は、特定のチェックポイントを一覧表示、検査、再開するための API を提供します。以下のような 2 つの Executor を順次実行するシンプルなパイプラインを構築して実験してみましょう。

1. 入力テキストを大文字化して次の処理へ渡す
1. 受け取ったテキストを反転させて、ワークフローを完了させる


In [ ]:
# 利用可能なチェックポイントの一覧
checkpoints = await checkpoint_storage.list_checkpoints()

# チェックポイント情報を表示する
for checkpoint in checkpoints:
    summary = RequestInfoExecutor.checkpoint_summary(checkpoint)
    print(f"Checkpoint {summary.checkpoint_id}: iteration={summary.iteration_count}")
    print(f"  Shared state: {checkpoint.shared_state}")
    print(f"  Executor states: {list(checkpoint.executor_states.keys())}")
    print(f"  Messages: {checkpoint.messages}")


In [ ]:
async def checkpoint_resume_example():
    # 特定のチェックポイントから再開する
    if checkpoints:
        chosen_checkpoint_id = checkpoints[0].checkpoint_id

        # 新しいワークフローインスタンスを作成し、再開する
        new_workflow = create_workflow(checkpoint_storage)
        async for event in new_workflow.run_stream_from_checkpoint(
            chosen_checkpoint_id,
            checkpoint_storage=checkpoint_storage
        ):
            print(f"Resumed event: {event}")

In [ ]:
await checkpoint_resume_example()

**注目👀ポイント:**

- `.with_checkpointing()` により、各スーパーステップ終了時に自動的に状態が保存される
- `FileCheckpointStorage` がチェックポイントを JSON 形式でディスク(`./checkpoints`)に保存


**再開の仕組み:**
- `run_stream_from_checkpoint` が指定されたチェックポイントIDから状態を復元
- 保存されていたメッセージキュー、Executor 状態、イテレーション数がすべて復元される
- 中断した地点から処理が継続される


まず、1 番目の`upper` Executor を実行し、その状態をチェックポイントとして JSON で保存しておきます。その後、`FileCheckpointStorage` からロードしたチェックポイントをワークフローにロードして `run_stream_from_checkpoint` することで 1 番目の実行をスキップして 2番目の `reverse` から再開することができます。


### 高度なチェックポイント機能
Human-in-the-Loop 統合を使用したチェックポイント:

チェックポイント処理は、人間が入力するためにワークフローを一時停止し、後で再開できるように、ループ内の人間のワークフローとシームレスに機能します。




In [ ]:
# Assume we have workflow, checkpoint_id, and checkpoint_storage from previous examples
async def resume_with_responses_example():
    # 事前に用意された人間の応答で再開する
    responses = {"request_id_123": "approved"}
    checkpoint_id = "2b981805-60e9-4d03-9623-738c2140b524"

    async for event in workflow.run_stream_from_checkpoint(
        checkpoint_id,
        checkpoint_storage=checkpoint_storage,
        responses=responses  # 人間のレスポンスを提供
    ):
        print(f"Event: {event}")

In [ ]:
await resume_with_responses_example()